<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc" style="margin-top: 1em;"><ul class="toc-item"><li><span><a href="#System-Information" data-toc-modified-id="System-Information-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>System Information</a></span></li><li><span><a href="#Packages" data-toc-modified-id="Packages-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Packages</a></span></li><li><span><a href="#Dataset" data-toc-modified-id="Dataset-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Dataset</a></span></li><li><span><a href="#Solutions" data-toc-modified-id="Solutions-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Solutions</a></span><ul class="toc-item"><li><span><a href="#Solution-1---Naive" data-toc-modified-id="Solution-1---Naive-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Solution 1 - Naive</a></span></li><li><span><a href="#Solution-2---Using-Performance-Tips" data-toc-modified-id="Solution-2---Using-Performance-Tips-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>Solution 2 - Using Performance Tips</a></span><ul class="toc-item"><li><span><a href="#Avoid-global-variables" data-toc-modified-id="Avoid-global-variables-4.2.1"><span class="toc-item-num">4.2.1&nbsp;&nbsp;</span>Avoid global variables</a></span></li><li><span><a href="#Pre-allocating-outputs" data-toc-modified-id="Pre-allocating-outputs-4.2.2"><span class="toc-item-num">4.2.2&nbsp;&nbsp;</span>Pre-allocating outputs</a></span></li></ul></li><li><span><a href="#Solution-3---Use-A-Profiler" data-toc-modified-id="Solution-3---Use-A-Profiler-4.3"><span class="toc-item-num">4.3&nbsp;&nbsp;</span>Solution 3 - Use A Profiler</a></span></li><li><span><a href="#Solution-4---Simpler-Data-Structure" data-toc-modified-id="Solution-4---Simpler-Data-Structure-4.4"><span class="toc-item-num">4.4&nbsp;&nbsp;</span>Solution 4 - Simpler Data Structure</a></span></li><li><span><a href="#Solution-5---Performance-Annotation" data-toc-modified-id="Solution-5---Performance-Annotation-4.5"><span class="toc-item-num">4.5&nbsp;&nbsp;</span>Solution 5 - Performance Annotation</a></span></li><li><span><a href="#Solution-6---No-Iterators" data-toc-modified-id="Solution-6---No-Iterators-4.6"><span class="toc-item-num">4.6&nbsp;&nbsp;</span>Solution 6 - No Iterators</a></span></li><li><span><a href="#Solution-7---Rephrasing-The-Problem" data-toc-modified-id="Solution-7---Rephrasing-The-Problem-4.7"><span class="toc-item-num">4.7&nbsp;&nbsp;</span>Solution 7 - Rephrasing The Problem</a></span></li><li><span><a href="#Solution-8---Task-Parallelism" data-toc-modified-id="Solution-8---Task-Parallelism-4.8"><span class="toc-item-num">4.8&nbsp;&nbsp;</span>Solution 8 - Task Parallelism</a></span></li></ul></li><li><span><a href="#Appendix" data-toc-modified-id="Appendix-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Appendix</a></span></li></ul></div>

# System Information

In [1]:
versioninfo(verbose=true)

Julia Version 1.3.1
Commit 2d5741174c (2019-12-30 21:36 UTC)
Platform Info:
  OS: Linux (x86_64-pc-linux-gnu)
      Ubuntu 18.04.3 LTS
  uname: Linux 4.15.0-1051-aws #53-Ubuntu SMP Wed Sep 18 13:35:53 UTC 2019 x86_64 x86_64
  CPU: AMD EPYC 7571: 
              speed         user         nice          sys         idle          irq
       #1  2199 MHz     172703 s          0 s      11525 s    3137793 s          0 s
       #2  2199 MHz     159759 s          0 s      13054 s    3148813 s          0 s
       #3  2199 MHz     168185 s      29119 s      14715 s    3107941 s          0 s
       #4  2199 MHz     157599 s      26896 s      15503 s    3117293 s          0 s
       
  Memory: 15.526390075683594 GB (12023.19921875 MB free)
  Uptime: 33307.0 sec
  Load Avg:  0.2626953125  0.1259765625  0.23095703125
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-6.0.1 (ORCJIT, znver1)
Environment:
  JULIA_NUM_THREADS = 4
  LD_LIBRARY_PATH = /usr/lib64/openmpi/lib/:/usr/local/cuda/lib64:/usr/loc

# Packages

In [2]:
using Distributions
using DataFrames
using DataStructures
using BenchmarkTools

# Dataset

Create the dataset:

In [3]:
const N = 6_000_000

6000000

In [4]:
U = DiscreteUniform(1, N)
small_cap = 'a':'z'
large_cap = 'A':'Z'

col1 = rand(large_cap, N)
col2 = rand(small_cap, N)
col3 = rand(U, N)

df = DataFrame(col1 = col1, col2 = col2, col3 = col3)

first(df, 5)

,col1,col2,col3
,Char,Char,Int64
1,'F','m',323255
2,'R','u',973180
3,'J','u',3329283
4,'R','t',4186964
5,'I','s',3371756


# Solutions

## Solution 1 - Naive

In [5]:
results = counter(Pair{Char,Char})

@btime for (c1, c2, c3) in eachrow($df)
    key, val = c1 => c2, c3
    results[key] = results[key] + val
end

  11.389 s (89997412 allocations: 1.70 GiB)


Without timing so we can check that results are identical with other methods:

In [6]:
results1 = counter(Pair{Char,Char})

for (c1, c2, c3) in eachrow(df)
    key, val = c1 => c2, c3
    results1[key] = results1[key] + val
end

## Solution 2 - Using Performance Tips

Based on the [Performance Tips](https://docs.julialang.org/en/v1/manual/performance-tips/index.html) page, we can try:

### Avoid global variables

In [7]:
function f(data)
    results = counter(Pair{Char, Char})
    
    for (c1, c2, c3) in eachrow(data)
        key, val = c1 => c2, c3
        results[key] = results[key] + val
    end
    results
end
;

In [8]:
@btime f($df);

  11.001 s (89996755 allocations: 1.70 GiB)


### Pre-allocating outputs

In [9]:
function f(data)
    results = counter(Pair{Char, Char})

    for (c1, c2) in Iterators.product('A':'Z', 'a':'z')
        key = Pair(c1, c2)
        results[key] = 0
    end
    
    for (c1, c2, c3) in eachrow(data)
        key, val = c1 => c2, c3
        results[key] = results[key] + val
    end

    results
end
;

In [10]:
@btime f($df);

  11.203 s (89996755 allocations: 1.70 GiB)


## Solution 3 - Use A Profiler

Save the following code into a file named `scratch.jl` and execute it in a REPL inside [Juno](http://docs.junolab.org/stable/):

```julia
using Distributions
using DataFrames
using DataStructures
using BenchmarkTools

const N = 6_000_000

U = DiscreteUniform(1, N)
small_cap = 'a':'z'
large_cap = 'A':'Z'

col1 = rand(large_cap, N)
col2 = rand(small_cap, N)
col3 = rand(U, N)

df = DataFrame(col1 = col1, col2 = col2, col3 = col3)

function f(data)
    results = counter(Pair{Char, Char})

    for (c1, c2, c3) in eachrow(data)
        key, val = c1 => c2, c3
        results[key] = results[key] + val
    end
    results
end

using Profile
Profile.init(delay=0.5)
Juno.@profiler f(df);
```

<img src="https://i.ibb.co/kHmpgD3/Untitled.png" alt="Untitled" border="0">

This gives:

<img src="https://i.ibb.co/8dJ6bQt/Untitled.png" alt="Untitled" border="0">

If we don't depend on data frames:

In [11]:
function f(data)
    results = counter(Pair{Char, Char})
    
    for (c1, c2, c3) in data
        key, val = c1 => c2, c3
        results[key] = results[key] + val
    end
    results
end
;

In [12]:
data = zip(df[!, :col1], df[!, :col2], df[!, :col3])
typeof(data)

Base.Iterators.Zip{Tuple{Array{Char,1},Array{Char,1},Array{Int64,1}}}

In [13]:
@btime f($data);

  187.673 ms (19 allocations: 92.33 KiB)


<img src="https://i.ibb.co/PgCYDh2/Untitled.png" alt="Untitled" border="0">

## Solution 4 - Simpler Data Structure

How to encode a pair of characters e.g. ("A", 'b') as integers:

In [14]:
convert(UInt16, 'A')

0x0041

In [15]:
convert(UInt16, 'b')

0x0062

In [16]:
convert(UInt16, 'A') << 8 | convert(UInt16, 'b')

0x4162

The maximum number of things this scheme can encode:

In [17]:
2^16

65536

A function to do the encoding:

In [18]:
g(x1, x2) = convert(UInt16, x1) << 8 | convert(UInt16, x2);

The solution:

In [19]:
function f(data)
    results = zeros(Int, 2^16)
    
    for (char1, char2, val) in data
        key = g(char1, char2)
        results[key] =  results[key] + val
    end
    results
end
;

In [20]:
@btime f($data);

  37.476 ms (2 allocations: 512.08 KiB)


## Solution 5 - Performance Annotation

In [21]:
function f(data)
    results = zeros(Int, 2^16)
    
    for (char1, char2, val) in data
        key = g(char1, char2)
        @inbounds results[key] =  results[key] + val
    end
    results
end
;

In [22]:
@btime f($data);

  33.819 ms (2 allocations: 512.08 KiB)


<img src="https://i.ibb.co/37Qbr2t/Untitled.png" alt="Untitled" border="0">

## Solution 6 - No Iterators

In [23]:
function f(col1, col2, col3)
    results = zeros(Int, 2^16)
    
    for i in eachindex(col1)
        key = g(col1[i], col2[i])
        @inbounds results[key] =  results[key] + col3[i]
    end
    results
end
;

In [24]:
@btime f($col1, $col2, $col3);

  26.786 ms (2 allocations: 512.08 KiB)


<img src="https://i.ibb.co/wRr814s/Untitled.png" alt="Untitled" border="0">

In [25]:
results =  f(col1, col2, col3)

for (k, v) in results1
    char1, char2 = k
    ind = g(char1, char2)
    @assert(results[ind] == v)
end

## Solution 7 - Rephrasing The Problem

In [26]:
@btime aggregate($df, $[:col1, :col2], $sum);

  233.226 ms (50062 allocations: 250.22 MiB)


## Solution 8 - Task Parallelism

Check out [this](https://julialang.org/blog/2019/07/multithreading/) blog post for details.

In [27]:
import Base.Threads.@spawn

In [28]:
chunk_dataframe(df, chunks = 4) = begin
    N = nrow(df)
    chunksize = Int(N / chunks)
    
    inds = [UnitRange((i - 1) * chunksize + 1, i * chunksize) for i in 1:chunks]
    [df[ind, :] for ind in inds]
end

chunk_dataframe (generic function with 2 methods)

In [29]:
count_data(df) = begin
    results = counter(Pair{Char, Char})
    
    for (c1, c2, c3) in eachrow(df)
        key, val = c1 => c2, c3
        results[key] = results[key] + val
    end
    results
end

count_data (generic function with 1 method)

In [30]:
function f(df, chunks = 4)
    
    df_chunks = chunk_dataframe(df, chunks)

    [@spawn count_data(df_chunk) for df_chunk in df_chunks] .|> 
        fetch |>
        x -> foldl(merge!, x)

end
;

In [31]:
@btime f($df, $4);

  12.611 s (89990352 allocations: 1.79 GiB)


In [32]:
@btime f($df, $2);

  11.668 s (89994663 allocations: 1.79 GiB)


# Appendix

Check that Task Parallelism would have worked if we had more expensive computation to perform.

In [33]:
sample_df = df[1:600_000, :];

Dummy function to represent some complex calculation:

In [34]:
function complicated_calculation!(results, key, value)
    Libc.systemsleep(1e-5)
    results[key] = results[key] + value
end;

If we process each row of the dataframe sequentially:

In [35]:
function f1(data)
    results = counter(Pair{Char, Char})
    
    for (c1, c2, c3) in eachrow(data)
        key, val = c1 => c2, c3
        complicated_calculation!(results, key, val)
    end
    results
end
;

In [36]:
results1_app = @btime f1(sample_df);

  48.349 s (8398385 allocations: 164.86 MiB)


Processing the rows in parallel will result in significant speedup:

In [37]:
complicated_count_data(df) = begin
    results = counter(Pair{Char, Char})
    
    for (c1, c2, c3) in eachrow(df)
        key, val = c1 => c2, c3
        complicated_calculation!(results, key, val)
    end
    results
end
;

In [38]:
function f2(df, chunks = 4)
    
    df_chunks = chunk_dataframe(df, chunks)

    [@spawn complicated_count_data(df_chunk) for df_chunk in df_chunks] .|> 
        fetch |>
        x -> foldl(merge!, x)

end
;

In [39]:
results2_app = @btime f2(sample_df, 4);

  12.750 s (8394009 allocations: 174.23 MiB)


Check results are identical:

In [40]:
results1_app == results2_app

true